In [21]:
import great_expectations as gx

In [22]:
from great_expectations.checkpoint import Checkpoint, LegacyCheckpoint
from great_expectations.checkpoint.types.checkpoint_result import CheckpointResult
from great_expectations.core import ExpectationSuiteValidationResult
from great_expectations.core.batch import BatchRequest, RuntimeBatchRequest

In [23]:
context = gx.get_context()

`batch_request` for retrieving `event_names` table

In [24]:
batch_request: BatchRequest = BatchRequest(
    **{
        "datasource_name": "my_datasource",
        "data_connector_name": "my_sql_data_connector",
        "data_asset_name": "event_names",
    }
)


`checkpoint` configuration 

In [25]:
checkpoint_config: dict = {
    "class_name": "Checkpoint",
    "name": "my_checkpoint",
    "config_version": 1,
    "module_name": "great_expectations.checkpoint",
    "run_name_template": "%Y-%M-foo-bar-template",
    "expectation_suite_name": "visitors_exp",
    "action_list": [
        {
            "name": "store_validation_result",
            "action": {
                "class_name": "StoreValidationResultAction",
            },
        },
        {
            "name": "store_evaluation_params",
            "action": {
                "class_name": "StoreEvaluationParametersAction",
            },
        },
        {
            "name": "update_data_docs",
            "action": {
                "class_name": "UpdateDataDocsAction",
            },
        },
    ],
    "validations": [
        {
            "batch_request": batch_request,
        }
    ],
}

In [26]:
context.add_checkpoint(**checkpoint_config)

{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction"
      }
    }
  ],
  "batch_request": {},
  "class_name": "Checkpoint",
  "config_version": 1,
  "evaluation_parameters": {},
  "expectation_suite_name": "visitors_exp",
  "module_name": "great_expectations.checkpoint",
  "name": "my_checkpoint",
  "profilers": [],
  "run_name_template": "%Y-%M-foo-bar-template",
  "runtime_configuration": {},
  "validations": [
    {
      "batch_request": {
        "datasource_name": "my_datasource",
        "data_connector_name": "my_sql_data_connector",
        "data_asset_name": "event_names"
      }
    }
  ]
}

# Example 1: 
* `unexpected_index_column_names` not configured.

In [7]:
results = context.run_checkpoint(checkpoint_name="my_checkpoint")

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

In [8]:
evrs = results.list_validation_results()

In [9]:
evrs[0]["results"][0]["result"]

{'element_count': 6,
 'unexpected_count': 3,
 'unexpected_percent': 50.0,
 'partial_unexpected_list': ['user_signup', 'purchase', 'download'],
 'missing_count': 0,
 'missing_percent': 0.0,
 'unexpected_percent_total': 50.0,
 'unexpected_percent_nonmissing': 50.0,
 'partial_unexpected_counts': [{'value': 'download', 'count': 1},
  {'value': 'purchase', 'count': 1},
  {'value': 'user_signup', 'count': 1}]}

In [10]:
context.open_data_docs()

# Example 2:
* Single `unexpected_index_column_names` configured.

In [11]:
result_format: dict = {
    "result_format": "COMPLETE",
    "unexpected_index_column_names": ["event_id"],
}


In [12]:
results: CheckpointResult = context.run_checkpoint(
    checkpoint_name="my_checkpoint", result_format=result_format
)

Calculating Metrics:   0%|          | 0/13 [00:00<?, ?it/s]

In [13]:
evrs = results.list_validation_results()

In [14]:
evrs[0]["results"][0]["result"]

{'element_count': 6,
 'unexpected_count': 3,
 'unexpected_percent': 50.0,
 'partial_unexpected_list': ['user_signup', 'purchase', 'download'],
 'unexpected_index_column_names': ['event_id'],
 'missing_count': 0,
 'missing_percent': 0.0,
 'unexpected_percent_total': 50.0,
 'unexpected_percent_nonmissing': 50.0,
 'partial_unexpected_index_list': [{'event_type': 'user_signup',
   'event_id': 3},
  {'event_type': 'purchase', 'event_id': 4},
  {'event_type': 'download', 'event_id': 5}],
 'partial_unexpected_counts': [{'value': 'download', 'count': 1},
  {'value': 'purchase', 'count': 1},
  {'value': 'user_signup', 'count': 1}],
 'unexpected_list': ['user_signup', 'purchase', 'download'],
 'unexpected_index_list': [{'event_type': 'user_signup', 'event_id': 3},
  {'event_type': 'purchase', 'event_id': 4},
  {'event_type': 'download', 'event_id': 5}],
 'unexpected_index_query': "SELECT event_type, event_id \nFROM event_names \nWHERE event_type IS NOT NULL AND (event_type NOT IN ('page_load', '

In [15]:
context.open_data_docs()

# Example 3:
* Multiple `unexpected_index_column_names` configured.

In [16]:
result_format: dict = {
    "result_format": "COMPLETE",
    "unexpected_index_column_names": ["event_id", "visit_id"],
}


In [17]:
results: CheckpointResult = context.run_checkpoint(
    checkpoint_name="my_checkpoint", result_format=result_format
)

Calculating Metrics:   0%|          | 0/13 [00:00<?, ?it/s]

In [18]:
evrs = results.list_validation_results()

In [19]:
evrs[0]["results"][0]["result"]

{'element_count': 6,
 'unexpected_count': 3,
 'unexpected_percent': 50.0,
 'partial_unexpected_list': ['user_signup', 'purchase', 'download'],
 'unexpected_index_column_names': ['event_id', 'visit_id'],
 'missing_count': 0,
 'missing_percent': 0.0,
 'unexpected_percent_total': 50.0,
 'unexpected_percent_nonmissing': 50.0,
 'partial_unexpected_index_list': [{'event_type': 'user_signup',
   'event_id': 3,
   'visit_id': 1470387700},
  {'event_type': 'purchase', 'event_id': 4, 'visit_id': 1470438716},
  {'event_type': 'download', 'event_id': 5, 'visit_id': 1470420524}],
 'partial_unexpected_counts': [{'value': 'download', 'count': 1},
  {'value': 'purchase', 'count': 1},
  {'value': 'user_signup', 'count': 1}],
 'unexpected_list': ['user_signup', 'purchase', 'download'],
 'unexpected_index_list': [{'event_type': 'user_signup',
   'event_id': 3,
   'visit_id': 1470387700},
  {'event_type': 'purchase', 'event_id': 4, 'visit_id': 1470438716},
  {'event_type': 'download', 'event_id': 5, 'visi

In [20]:
context.open_data_docs()